# Пример использования

In [1]:
%pip install torch torchvision torchaudio
%pip install pandas
%pip install sentencepiece
%pip install transformers
%pip install datasets
%pip install seqeval


DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
You should consider upgrading via the '/opt/homebrew/opt/python@3.9/bin/python3.9 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installa

In [2]:
import pandas as pd
from datasets import Dataset
from datasets import load_metric
from transformers import AutoTokenizer
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
from transformers import DataCollatorForTokenClassification
import torch
import numpy as np


/opt/homebrew/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
entity_classes = [
    '0',
    'days_code',
    'last_name',
    'airline_name',
    'round_trip',
    'flight_number',
    'fromloc.state_code',
    'toloc.country_name',
    'connect',
    'transport_type',
    'airport_code',
    'toloc.city_name',
    'people',
    'fromloc.city_name',
    'cost_relative',
    'flight_time',
    'toloc.airport_name',
    'city_name',
    'mod',
    'or',
    'date',
    'toloc.airport_code',
    'flight_stop',
    'meal',
    'meal_description',
    'flight_mod',
    'fromloc.state_name',
    'flight_days',
    'airport_name',
    'meal_code',
    'state_name',
    'time',
    'fromloc.airport_code',
    'state_code',
    'aircraft_code',
    'restriction_code',
    'stoploc.city_name',
    'fare_amount',
    'fromloc.airport_name',
    'economy',
    'first_name',
    'airline_code',
    'class_type',
    'time_relative',
    'toloc.state_name',
    'toloc.state_code',
    'fare_basis_code',
    'period_mod']


In [4]:
tokenizer = AutoTokenizer.from_pretrained(
    '../ner/models/distilbert/five_epochs/'
)

paragraph = "My name is Torie Lindsey and I need to reschedule a table that I booked tonight at 07:45PM because some people are unable to make it."
tokens = tokenizer(paragraph)
torch.tensor(tokens['input_ids']).unsqueeze(0).size()

model = AutoModelForTokenClassification.from_pretrained(
    '../ner/models/distilbert/five_epochs/', num_labels=len(entity_classes)
)


In [5]:
predictions = model.forward(input_ids=torch.tensor(tokens['input_ids']).unsqueeze(
    0), attention_mask=torch.tensor(tokens['attention_mask']).unsqueeze(0))
predictions = torch.argmax(predictions.logits.squeeze(), axis=1)
predictions = [entity_classes[i] for i in predictions]

words = tokenizer.batch_decode(tokens['input_ids'])
print(pd.DataFrame({'ner': predictions, 'words': words}))


           ner    words
0            0    [CLS]
1            0       my
2            0     name
3            0       is
4   first_name     tori
5   first_name      ##e
6    last_name  lindsey
7            0      and
8            0        i
9            0     need
10           0       to
11           0      res
12           0   ##ched
13           0    ##ule
14           0        a
15           0    table
16           0     that
17           0        i
18           0   booked
19        date  tonight
20           0       at
21        time       07
22        time        :
23        time       45
24        time     ##pm
25           0  because
26           0     some
27           0   people
28           0      are
29           0   unable
30           0       to
31           0     make
32           0       it
33           0        .
34           0    [SEP]
